Trainnig U-Net model

In [ ]:
# Installing libraries
!pip install segmentation-models
!pip install tensorflow==2.1.0
!pip install keras==2.3.1
!pip install h5py==2.10.0

In [ ]:
# Importing libraries
%matplotlib inline
import glob
import cv2
import random
import os
import numpy as np
from matplotlib import pyplot as plt
%env SM_FRAMEWORK=tf.keras
import tensorflow as tf
import segmentation_models as sm

In [ ]:
# Loading images and masks
from google.colab import drive
drive.mount('/content/drive')

img_augmented_path="/content/drive/MyDrive/Thigh_segmentation/Train/Images"
msk_augmented_path="/content/drive/MyDrive/Thigh_segmentation/Train/Labels"

n_classes=9 # Number of classes for segmentation

# Capturing training image info as a list
train_images = []

for directory_path in sorted(glob.glob(img_augmented_path)):
    for img_path in sorted(glob.glob(os.path.join(directory_path, "*.tif"))):
        img = cv2.imread(img_path, 1)       
        train_images.append(img)
       
# Converting list to array for deep learning processing        
train_images = np.array(train_images)

# Capturing mask/label info as a list
train_masks = [] 
for directory_path in sorted(glob.glob(msk_augmented_path)):
    for mask_path in sorted(glob.glob(os.path.join(directory_path, "*.tif"))):
        mask = cv2.imread(mask_path, 0)
        train_masks.append(mask)
        
# Converting list to array   
train_masks = np.array(train_masks)

In [ ]:
# Checking the size and dimentions of arrays 
print(train_images.shape)
print(train_masks.shape)
# Sanity check for random images and correspounding masks
image_x = random.randint(0, (train_images.shape)[0])
plt.imshow(train_images[image_x])
print (image_x)
plt.show()
plt.imshow((train_masks[image_x]))
plt.show()

In [ ]:
# Encoding labels
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = train_masks.shape
train_masks_reshaped = train_masks.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)
print(train_masks_encoded_original_shape.shape)
print(train_masks_reshaped_encoded.shape)
np.unique(train_masks_encoded_original_shape)

In [ ]:
# Adding a dimension to masks to match images (Read in RGB)
train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=3) 

#Create a subset of data for validation (10%)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_images, train_masks_input, test_size = 0.10, random_state = 0)
print("Class values in the dataset are ... ", np.unique(y_train))

# Reformating classes
from tensorflow.keras.utils import to_categorical
train_masks_cat = to_categorical(y_train, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes)) 
test_masks_cat = to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))

In [ ]:
# Setting parameters
activation='softmax'
LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)
total_loss = sm.losses.categorical_focal_dice_loss
total_loss = sm.losses.bce_jaccard_loss
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), 'accuracy']

In [ ]:
### Model
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

# preprocess input
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)

In [ ]:
# Defining model
model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)
# Compiling keras model with defined optimozer, loss and metrics
model.compile(optimizer=optim, loss=total_loss, metrics=metrics)
print(model.summary())

In [ ]:
# Fitting model

history=model.fit(X_train, 
          y_train_cat,
          batch_size=10, 
          epochs=100,
          verbose=1,
          validation_data=(X_test, y_test_cat))


In [ ]:
# Saving the model
model.save('/content/drive/MyDrive/Thigh_segmentation/thigh_segmentation.h5')

In [ ]:
#Evaluate the model accuracy
_, acc, *is_anything_else_being_returned  = model.evaluate(X_test, y_test_cat)
print(model.evaluate(X_test, y_test_cat))
print("Accuracy is = ", (acc * 100.0), "%")
model.evaluate(X_test, y_test_cat)

In [ ]:
# Plot the training and validation accuracy and loss at each epoch
print(model.metrics_names)
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
print(model.metrics_names)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'y', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
#Calculating total IOU
y_pred=model.predict(X_test)
y_pred_argmax=np.argmax(y_pred, axis=3)
#Using built in keras function
from tensorflow.keras.metrics import MeanIoU
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(y_test[:,:,:,0], y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())

In [ ]:
# Calculating I0U for each class on validation dataset
values = np.array(IOU_keras.get_weights()).reshape(n_classes, n_classes)
class0_IoU = values[0,0]/(values[0,0] + values[0,1] + values[0,2] + values[0,3] + values[0,4] + values[0,5] + values[0,6] + values[0,7] + values[0,8] + values[1,0]+ values[2,0]+ values[3,0] + values[4,0] + values[5,0] + values[6,0] + values[7,0] + values[8,0])
class1_IoU = values[1,1]/(values[1,1] + values[1,0] + values[1,2] + values[1,3] + values[1,4] + values[1,5] + values[1,6] + values[1,7] + values[1,8] + values[0,1]+ values[2,1]+ values[3,1] + values[4,1] + values[5,1] + values[6,1] + values[7,1] + values[8,1])
class2_IoU = values[2,2]/(values[2,2] + values[2,0] + values[2,1] + values[2,3] + values[2,4] + values[2,5] + values[2,6] + values[2,7] + values[2,8] + values[0,2]+ values[1,2]+ values[3,2] + values[4,2] + values[5,2] + values[6,2] + values[7,2] + values[8,2])
class3_IoU = values[3,3]/(values[3,3] + values[3,0] + values[3,1] + values[3,2] + values[3,4] + values[3,5] + values[3,6] + values[3,7] + values[3,8] + values[0,3]+ values[1,3]+ values[2,3] + values[4,3] + values[5,3] + values[6,3] + values[7,3] + values[8,3])
class4_IoU = values[4,4]/(values[4,4] + values[4,0] + values[4,1] + values[4,2] + values[4,3] + values[4,5] + values[4,6] + values[4,7] + values[4,8] + values[0,4]+ values[1,4]+ values[2,4] + values[3,4] + values[5,4] + values[6,4] + values[7,4] + values[8,4])
class5_IoU = values[5,5]/(values[5,5] + values[5,0] + values[5,1] + values[5,2] + values[5,3] + values[5,4] + values[5,6] + values[5,7] + values[5,8] + values[0,5]+ values[1,5]+ values[2,5] + values[3,5] + values[4,5] + values[6,5] + values[7,5] + values[8,5])
class6_IoU = values[6,6]/(values[6,6] + values[6,0] + values[6,1] + values[6,2] + values[6,3] + values[6,4] + values[6,5] + values[6,7] + values[6,8] + values[0,6]+ values[1,6]+ values[2,6] + values[3,6] + values[4,6] + values[5,6] + values[7,6] + values[8,6])
class7_IoU = values[7,7]/(values[7,7] + values[7,0] + values[7,1] + values[7,2] + values[7,3] + values[7,4] + values[7,5] + values[7,6] + values[7,8] + values[0,7]+ values[1,7]+ values[2,7] + values[3,7] + values[4,7] + values[5,7] + values[6,7] + values[8,7])
class8_IoU = values[8,8]/(values[8,8] + values[8,0] + values[8,1] + values[8,2] + values[8,3] + values[8,4] + values[8,5] + values[8,6] + values[8,7] + values[0,8]+ values[1,8]+ values[2,8] + values[3,8] + values[4,8] + values[5,8] + values[6,8] + values[7,8])


print("IoU for class0 (Background) is: ", class0_IoU)
print("IoU for class1 (Medulla) is: ", class1_IoU)
print("IoU for class2 (Bone) is: ", class2_IoU)
print("IoU for class3 (SCF) is: ", class3_IoU)
print("IoU for class4 (Quadriceps) is: ", class4_IoU)
print("IoU for class5 (Flexors) is: ", class5_IoU)
print("IoU for class6 (Adductors) is: ", class6_IoU)
print("IoU for class7 (Sartorius) is: ", class7_IoU)
print("IoU for class8 (IMAT) is: ", class8_IoU)


In [ ]:
# Predicting on random images
test_img_number = random.randint(0, len(X_test)-1)
test_img = X_test[test_img_number]
ground_truth = y_test[test_img_number]
test_img_input = np.expand_dims(test_img, 0)
prediction = (model.predict(test_img_input))
predicted_img=np.argmax(prediction, axis=3)[0,:,:]
plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img[:,:,0], cmap='gray')
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth[:,:,0], cmap='jet')
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(predicted_img, cmap='jet')
plt.show()

Testing the model on holdout test set

In [ ]:
# Test on independent data

img_augmented_path="/content/drive/MyDrive/Thigh_segmentation/Test/Images"
msk_augmented_path="/content/drive/MyDrive/Thigh_segmentation/Test/Labels"

# Performing same formatting and preprocessing 
n_classes=9
test_images_ext = []
for directory_path in sorted(glob.glob(img_augmented_path)):
    for img_path in sorted(glob.glob(os.path.join(directory_path, "*.tif"))):
        img = cv2.imread(img_path, 1)       
        test_images_ext.append(img)
test_images_ext = np.array(test_images_ext)
test_masks_ext = [] 
for directory_path in sorted(glob.glob(msk_augmented_path)):
    for mask_path in sorted(glob.glob(os.path.join(directory_path, "*.tif"))):
        mask = cv2.imread(mask_path, 0)       
        #mask = cv2.resize(mask, (SIZE_Y, SIZE_X), interpolation = cv2.INTER_NEAREST)  #Otherwise ground truth changes due to interpolation
        test_masks_ext.append(mask)
test_masks_ext = np.array(test_masks_ext)

In [ ]:
# Performing sanity check on loaded images and masks
print(test_images_ext.shape)
print(test_masks_ext.shape)
image_x = random.randint(0, (test_images_ext.shape)[0])
#print((test_images_ext.shape)[0])
plt.imshow(test_images_ext[image_x])
print (image_x)
plt.show()
plt.imshow((test_masks_ext[image_x]))
plt.show()

In [ ]:
# Encoding labels
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = test_masks_ext.shape
test_masks_ext_reshaped = test_masks_ext.reshape(-1,1)
test_masks_ext_reshaped_encoded = labelencoder.fit_transform(test_masks_ext_reshaped)
test_masks_ext_encoded_original_shape = test_masks_ext_reshaped_encoded.reshape(n, h, w)

print(test_masks_ext_encoded_original_shape.shape)
print(test_masks_ext_reshaped_encoded.shape)
np.unique(test_masks_ext_encoded_original_shape)
X_test_ext = test_images_ext
X_test_ext = preprocess_input(X_test_ext)
y_test_ext = np.expand_dims(test_masks_ext_encoded_original_shape, axis=3)

# Reformating test masks
from tensorflow.keras.utils import to_categorical
test_masks_ext_cat = to_categorical(y_test_ext, num_classes=n_classes)
y_test_ext_cat = test_masks_ext_cat.reshape((y_test_ext.shape[0], y_test_ext.shape[1], y_test_ext.shape[2], n_classes))

In [ ]:
# IOU assessment
print(X_test_ext.shape)
y_pred_ext=model.predict(X_test_ext[:,:,:])
y_pred_ext_argmax=np.argmax(y_pred_ext, axis=3)
# Using built in keras function
from tensorflow.keras.metrics import MeanIoU
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(y_test_ext[:,:,:,0], y_pred_ext_argmax)
print("Mean IoU =", IOU_keras.result().numpy())

In [ ]:
EPS = 1e-12
def get_iou( gt , pr , n_classes ):
    class_wise = np.zeros(n_classes)
    for cl in range(n_classes):
        intersection = np.sum(( gt == cl )*( pr == cl ))
        union = np.sum(np.maximum( ( gt == cl ) , ( pr == cl ) ))
        iou = float(intersection)/( union + EPS )
        class_wise[ cl ] = iou
    return class_wise

ious = np.array(get_iou(y_test_ext[:,:,:,0], y_pred_ext_argmax,n_classes=n_classes))
print(ious)

# Removing background in calculation of mean IOU
print("Total  IoU near"  ,  np.mean(ious[1:9] ))

Prediction of the masks for all other OAI subjects, using trained model

In [ ]:
# Test on independent data

img_WholeOAI_path="/content/drive/MyDrive/Thigh_segmentation/WholeOAI/Images"

# Performing same formatting and preprocessing 
n_classes=9
img_list_np = []
for directory_path in sorted(glob.glob(img_WholeOAI_path)):
    for img_path in sorted(glob.glob(os.path.join(directory_path, "*.tif"))):
        img = cv2.imread(img_path, 1)       
        img_list_np.append(img)
img_list_np = np.array(img_list_np)

In [ ]:
LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)
total_loss = sm.losses.categorical_focal_dice_loss
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), 'accuracy']
activation='softmax'
n_classes=9
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

model = keras.models.load_model('/content/drive/MyDrive/Thigh_segmentation/thigh_segmentation.h5', compile=False)
model.compile(optimizer=optim, loss=total_loss, metrics=metrics)

print(img_list_np.shape)
image_x = random.randint(0, (img_list_np.shape)[0])
plt.imshow(img_list_np[image_x])
print (image_x)
plt.show()

In [ ]:
X_predict = img_list_np
X_predict = preprocess_input(X_predict)

y_predict = model.predict(X_predict[:,:,:])
y_predict_argmax=np.argmax(y_predict, axis=3)

In [ ]:
#Predict on a few images
test_img_number = random.randint(0, len(X_predict)-1)
print(test_img_number)
test_img = X_predict[test_img_number]
test_img_input = np.expand_dims(test_img, 0)
prediction = (model.predict(test_img_input))
predicted_img=np.argmax(prediction, axis=3)[0,:,:]

plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img[:,:,0], cmap='gray')
plt.subplot(232)
plt.title('Prediction on test image')
plt.imshow(predicted_img, cmap='jet')
plt.show()

In [ ]:
np.save('/content/drive/MyDrive/Thigh_segmentation/y_predict_argmax_WholeOAI.npy',y_predict_argmax)